In [2]:
#0.Import Libraries & Basic Settings
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="torch")

import os, datetime
import numpy as np
import pandas as pd
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, classification_report, confusion_matrix
import joblib

# 既存モジュール
from Generate_Features_and_Labels import generate_labels_and_features, add_trade_weights
from LSTM_model_classify import LSTMClassifier, make_latest_lstm_window

#1.Set Parameters
# データ
symbol = 'ASTERUSDT'
csv_filename = 'Market_Bybit_ASTERUSDT_1min_20250901-20250926'
csv_path = f'01.data/{csv_filename}.csv'
model_min = 1

# ラベル生成
twap_forward_period = 60
twap_lookback_period = 60

# 損失ウェイト設定
weight_fee = 0.0011
weight_slip = 0.000
weight_quantile = 0.95
weight_gamma = 0.9995
weight_alpha = 0.9
weight_min = 0.0
weitght_max = 3.0
weight_sigma_scale = 0.1

# LSTM
LSTM_lookback   = 48
LSTM_hidden     = 128
LSTM_layers     = 2
LSTM_dropout    = 0.2
LSTM_num_classes= 2
LSTM_learnRate  = 1e-3
LSTM_gamma      = 0.985
LSTM_epochs     = 50
LSTM_train_ratio= 0.8
LSTM_val_frac   = 0.1
LSTM_patience   = 8
LSTM_min_delta  = 1e-3
LSTM_batch = 256
LSTM_seed = 18

# 特徴/ラベル
Features = ['O_log', 'H_log', 'L_log', 'C_log', 'V_log']
Label    = 'label'

# 保存
models_dir  = "./Models"
scaler_path = f'./scaler_{symbol}_{model_min}min.gz'   # ★足に合わせて命名
batch_size  = LSTM_batch
seed        = LSTM_seed

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Seed固定 ===
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

os.makedirs(models_dir, exist_ok=True)

#3.Utility Functions (only within ipynb)
def f3(x):
    try:
        if x is None: return "nan"
        if isinstance(x, float) and (np.isnan(x) or np.isinf(x)): return "nan"
        return f"{x:.3f}"
    except Exception:
        return "nan"

def make_sequences(X2d: np.ndarray, y1d: np.ndarray, w1d: np.ndarray, seq_len: int):
    if len(X2d) < seq_len:
        raise ValueError(f"データ不足: len={len(X2d)}, SEQ_LEN={seq_len}")
    Xs, ys, ws = [], [], []
    for i in range(seq_len-1, len(X2d)):
        Xs.append(X2d[i-seq_len+1:i+1])
        ys.append(y1d[i])
        ws.append(w1d[i])   # ラベルと同じ時点の重みを付与
    return np.stack(Xs), np.array(ys), np.array(ws)


def evaluate(model, loader, device, criterion=None, weighted=False):
    """
    loader: (X,y) または (X,y,w) のいずれか
    criterion:
      - None なら損失は計算しない
      - reduction='none' のCEを渡すと weighted=True で重み付き損失を計算
      - それ以外なら通常平均損失
    weighted: True のとき (X,y,w) なら重み付き平均損失で集計
    """
    model.eval()
    all_y, all_pred, all_prob = [], [], []
    total_loss = 0.0
    n_batches = 0

    with torch.no_grad():
        for batch in loader:
            if len(batch) == 3:
                Xb, yb, wb = batch
                wb = wb.to(device)
            else:
                Xb, yb = batch
                wb = None

            Xb = Xb.to(device); yb = yb.to(device)
            logits = model(Xb)

            # --- 損失 ---
            if criterion is not None:
                if weighted and wb is not None and getattr(criterion, 'reduction', 'mean') == 'none':
                    loss_vec = criterion(logits, yb)                 # (B,)
                    denom = torch.clamp(wb.sum(), min=1e-8)
                    loss = (loss_vec * wb).sum() / denom
                else:
                    # 通常（重み無視）の平均損失
                    loss = criterion(logits, yb)
                total_loss += float(loss.detach().cpu())
                n_batches += 1

            # --- 予測/メトリクス用 ---
            probs = torch.softmax(logits, dim=1)
            pred  = probs.argmax(dim=1)

            all_y.append(yb.cpu()); all_pred.append(pred.cpu()); all_prob.append(probs.cpu())

    all_y    = torch.cat(all_y).numpy()
    all_pred = torch.cat(all_pred).numpy()
    all_prob = torch.cat(all_prob).numpy()

    acc = (all_y == all_pred).mean()
    bal_acc, auc = float('nan'), float('nan')
    if len(np.unique(all_y)) >= 2:
        try:
            bal_acc = balanced_accuracy_score(all_y, all_pred)
        except: pass
        try:
            if all_prob.shape[1] == 2:
                auc = roc_auc_score(all_y, all_prob[:, 1])
            else:
                auc = roc_auc_score(all_y, all_prob, multi_class="ovr")
        except: pass

    avg_loss = total_loss / max(1, n_batches) if criterion is not None else float('nan')
    return avg_loss, acc, bal_acc, auc, all_y, all_pred, all_prob


#4.Read csv and generate features/labels
df_raw = pd.read_csv(csv_path)
df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'])

df_feat = generate_labels_and_features(df_raw, twap_forward_period, twap_lookback_period)
df_w = add_trade_weights(df_feat, fee=weight_fee, slip=weight_slip, rem_horizon=twap_forward_period, quantile=weight_quantile,
                         gamma=weight_gamma, alpha=weight_alpha, wmin=weight_min, wmax=weitght_max, sigma_scale=weight_sigma_scale)

use_cols = ['timestamp'] + Features + [Label, 'weight', 'weight_base', 'weight_rem']
data = df_w[use_cols].dropna().reset_index(drop=True)
data.head()

#5.Split / Standardize & Sequence
split_idx = int(len(data) * LSTM_train_ratio)
train_all = data.iloc[:split_idx].copy()
test_df   = data.iloc[split_idx:].copy()
val_idx   = int(len(train_all) * (1 - LSTM_val_frac))
train_df  = train_all.iloc[:val_idx].copy()
val_df    = train_all.iloc[val_idx:].copy()

# Standardize（特徴だけ）
scaler = StandardScaler()
X_train_2d = scaler.fit_transform(train_df[Features].values)
X_val_2d   = scaler.transform(val_df[Features].values)
X_test_2d  = scaler.transform(test_df[Features].values)

y_train_1d = train_df[Label].astype(int).values
y_val_1d   = val_df[Label].astype(int).values
y_test_1d  = test_df[Label].astype(int).values

w_train_1d = train_df['weight'].astype(float).values
w_val_1d   = val_df['weight'].astype(float).values
w_test_1d  = test_df['weight'].astype(float).values

# Save scaler
joblib.dump(scaler, scaler_path)
print("Saved scaler:", scaler_path)

# Sequences（重み込み）
X_train, y_train, w_train = make_sequences(X_train_2d, y_train_1d, w_train_1d, LSTM_lookback)
X_val,   y_val,   w_val   = make_sequences(X_val_2d,   y_val_1d,   w_val_1d,   LSTM_lookback)
X_test,  y_test,  w_test  = make_sequences(X_test_2d,  y_test_1d,  w_test_1d,  LSTM_lookback)

print("shapes →",
      "X_train", X_train.shape, "y_train", y_train.shape, "w_train", w_train.shape,
      "| X_val", X_val.shape,   "y_val",   y_val.shape,   "w_val",   w_val.shape,
      "| X_test", X_test.shape, "y_test",  y_test.shape,  "w_test",  w_test.shape)


#6.Tensor/Dataloader
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
w_train_t = torch.tensor(w_train, dtype=torch.float32)

X_val_t = torch.tensor(X_val, dtype=torch.float32)
y_val_t = torch.tensor(y_val, dtype=torch.long)
w_val_t = torch.tensor(w_val, dtype=torch.float32)

X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)
w_test_t = torch.tensor(w_test, dtype=torch.float32)

train_loader = DataLoader(TensorDataset(X_train_t, y_train_t, w_train_t), batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val_t,   y_val_t,   w_val_t),   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(TensorDataset(X_test_t,  y_test_t,  w_test_t),  batch_size=batch_size, shuffle=False)


#7.Model
model = LSTMClassifier(
    input_dim=len(Features),
    hidden_dim=LSTM_hidden,
    num_layers=LSTM_layers,
    num_classes=LSTM_num_classes,
    dropout=LSTM_dropout
).to(device)

criterion_ce = nn.CrossEntropyLoss(reduction='none')
optimizer = optim.Adam(model.parameters(), lr=LSTM_learnRate)
scheduler = ExponentialLR(optimizer, gamma=LSTM_gamma)

#8.Training
patience  = LSTM_patience
min_delta = LSTM_min_delta
no_improve = 0
best_val = None
best_state = None
best_epoch = None

def select_metric(val_balacc, val_acc):
    m = val_balacc
    if isinstance(m, float) and (np.isnan(m) or np.isinf(m)):
        m = val_acc
    return float(m)

use_amp = (device.type == "cuda")
scaler_amp = torch.amp.GradScaler("cuda") if use_amp else None

for epoch in range(1, LSTM_epochs + 1):
    model.train()
    total = 0.0

    for batch in train_loader:
        Xb, yb, wb = batch
        Xb = Xb.to(device); yb = yb.to(device); wb = wb.to(device)
        optimizer.zero_grad(set_to_none=True)

        if use_amp:
            with torch.amp.autocast("cuda"):
                logits = model(Xb)                             # (B, 2)
                loss_vec = criterion_ce(logits, yb)           # (B,)
                # 正規化：重みの和で割る（ゼロ分母回避）
                denom = torch.clamp(wb.sum(), min=1e-8)
                loss = (loss_vec * wb).sum() / denom
            scaler_amp.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler_amp.step(optimizer)
            scaler_amp.update()
        else:
            logits = model(Xb)
            loss_vec = criterion_ce(logits, yb)
            denom = torch.clamp(wb.sum(), min=1e-8)
            loss = (loss_vec * wb).sum() / denom
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        total += float(loss.detach().cpu())

    train_loss = total / max(1, len(train_loader))
    scheduler.step()

    criterion_eval = nn.CrossEntropyLoss(reduction='none')
    val_loss, val_acc, val_balacc, val_auc, *_ = evaluate(model, val_loader, device, criterion_eval, weighted=True)

    metric = select_metric(val_balacc, val_acc)

    improved = (best_val is None) or (metric > best_val + min_delta)


    if improved:
        best_val = metric
        best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        best_epoch = epoch
        no_improve = 0
    else:
        no_improve += 1

    if epoch % 5 == 0 or epoch == 1:
        print(f"[{epoch:03d}/{LSTM_epochs}] "
              f"train_loss {train_loss:.4f} | "
              f"val_loss {val_loss:.4f} | "
              f"val_acc {f3(val_acc)} | "
              f"val_bal_acc {f3(val_balacc)} | "
              f"val_auc {f3(val_auc)} | "
              f"best_metric {best_val:.4f} | "
              f"no_improve {no_improve}/{patience}")

    if no_improve >= patience:
        print(f"⏹️ Early stopped at epoch {epoch} (best at {best_epoch}, best_metric={best_val:.4f}).")
        break


# === Best restore（同じ） ===
if best_state is not None:
    model.load_state_dict(best_state)
    print(f"✅ Restored best validation model (epoch {best_epoch}, best_metric={best_val:.4f}).")

# === Save path: 未定義変数を使わない命名に修正 ===
ts = datetime.datetime.now().strftime("%y%m%d%H%M")
# 例1: ご希望の最小形式
model_path = os.path.join(models_dir, f"LSTM_classify_{symbol}_{ts}.pt")
# 例2: もう少し情報を持たせるなら（任意）
# model_path = os.path.join(models_dir, f"LSTM_classify_{symbol}_{model_min}m_lb{LSTM_lookback}_{ts}.pt")

torch.save({
    "epoch": best_epoch,
    "model_state_dict": model.state_dict(),
    "best_metric": best_val,
    "params": {
        "lookback": LSTM_lookback,
        "hidden": LSTM_hidden,
        "layers": LSTM_layers,
        "dropout": LSTM_dropout,
        "num_classes": LSTM_num_classes,
        "input_dim": len(Features),
        "features": Features,           # ★追加（任意）
        "scaler_path": scaler_path,     # ★追加（任意）
        "train_ratio": LSTM_train_ratio,# ★追加（任意）
    }
}, model_path)
print(f"💾 Model saved to {model_path}")

#9.Model Evaluation
criterion_eval = nn.CrossEntropyLoss()  # ★評価用に追加

criterion_eval = nn.CrossEntropyLoss(reduction='none')
test_loss, test_acc, test_balacc, test_auc, y_true, y_pred, y_prob = evaluate(model, test_loader, device, criterion_eval, weighted=True)

print(f"\n[Test] loss {test_loss:.4f} | acc {f3(test_acc)} | bal_acc {f3(test_balacc)} | auc {f3(test_auc)}")

print("\nClassification report (labels: 0=Short, 1=Long):")
print(classification_report(y_true, y_pred, digits=3, target_names=["Short(0)", "Long(1)"]))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved scaler: ./scaler_ASTERUSDT_1min.gz
shapes → X_train (6603, 48, 5) y_train (6603,) w_train (6603,) | X_val (692, 48, 5) y_val (692,) w_val (692,) | X_test (1801, 48, 5) y_test (1801,) w_test (1801,)
[001/50] train_loss 0.6462 | val_loss 0.5588 | val_acc 0.717 | val_bal_acc 0.702 | val_auc 0.768 | best_metric 0.7024 | no_improve 0/8
[005/50] train_loss 0.5442 | val_loss 0.5199 | val_acc 0.718 | val_bal_acc 0.714 | val_auc 0.797 | best_metric 0.7209 | no_improve 1/8
[010/50] train_loss 0.5337 | val_loss 0.5082 | val_acc 0.754 | val_bal_acc 0.732 | val_auc 0.802 | best_metric 0.7319 | no_improve 0/8
[015/50] train_loss 0.5320 | val_loss 0.5135 | val_acc 0.730 | val_bal_acc 0.727 | val_auc 0.801 | best_metric 0.7319 | no_improve 5/8
⏹️ Early stopped at epoch 18 (best at 10, best_metric=0.7319).
✅ Restored best validation model (epoch 10, best_metric=0.7319).
💾 Model saved to ./Models\LSTM_classify_

In [ ]:
#10. Inference / Build df_predicted
import torch
import joblib
import numpy as np
import pandas as pd
from typing import Optional

@torch.no_grad()
def run_inference_build_df(
    df_raw: pd.DataFrame,
    model_path: str,
    scaler_path: str,
    features: list,
    label_col: str,
    seq_len: int,
    device: Optional[torch.device] = None,
    batch_size: int = 1024,
    keep_prob: bool = False,   # Trueなら確信度も出力
) -> pd.DataFrame:
    """
    学習済みLSTM分類モデルをロードし、df_raw全体に対してスライディング推論。
    返り値は [timestamp, O,H,L,C,V, Pred, Label] (+ ProbLong 任意) を持つ df_predicted。
    """

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # --- 1) 特徴・ラベル再生成（学習時と同じ関数を利用） ---
    df_feat = generate_labels_and_features(df_raw.copy(), twap_forward_period, twap_lookback_period)

    # この時点で NaN が入る列を落として学習時と同様の整形
    use_cols = ['timestamp'] + features + [label_col]
    data = df_feat[use_cols].dropna().reset_index(drop=True)

    # --- 2) スケーラー読み込み & 標準化（特徴のみ） ---
    scaler: StandardScaler = joblib.load(scaler_path)
    X2d = scaler.transform(data[features].values)
    y1d = data[label_col].astype(int).values

    # --- 3) スライディング窓（学習時と整合のある作り） ---
    if len(X2d) < seq_len:
        raise ValueError(f"推論用データ不足: len={len(X2d)}, SEQ_LEN={seq_len}")

    # ここでは重み不要なのでダミーでOK
    def make_sequences_for_infer(X2d: np.ndarray, y1d: np.ndarray, seq_len: int):
        Xs, ys = [], []
        for i in range(seq_len-1, len(X2d)):
            Xs.append(X2d[i-seq_len+1:i+1])
            ys.append(y1d[i])
        return np.stack(Xs), np.array(ys)

    X_seq, y_seq = make_sequences_for_infer(X2d, y1d, seq_len)
    X_t = torch.tensor(X_seq, dtype=torch.float32, device=device)

    # --- 4) モデル読み込み ---
    ckpt = torch.load(model_path, map_location=device, weights_only=True)
    params = ckpt.get("params", {})
    input_dim  = params.get("input_dim", len(features))
    hidden_dim = params.get("hidden", LSTM_hidden)
    num_layers = params.get("layers", LSTM_layers)
    dropout    = params.get("dropout", LSTM_dropout)
    num_classes= params.get("num_classes", LSTM_num_classes)

    model = LSTMClassifier(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=num_classes,
        dropout=dropout
    ).to(device)
    model.load_state_dict(ckpt["model_state_dict"])
    model.eval()

    # --- 5) バッチ推論（確率 & 予測ラベル） ---
    preds = []
    prob1 = []  # クラス1(=Long) の確率
    for i in range(0, len(X_t), batch_size):
        xb = X_t[i:i+batch_size]
        logits = model(xb)                     # (B, num_classes)
        probs  = torch.softmax(logits, dim=1)  # (B, num_classes)
        pred   = probs.argmax(dim=1)           # (B,)
        preds.append(pred.cpu().numpy())
        if probs.shape[1] >= 2:
            prob1.append(probs[:, 1].cpu().numpy())
        else:
            # 2クラス想定だが念のため
            prob1.append(np.full(len(pred), np.nan))

    preds = np.concatenate(preds)
    prob1 = np.concatenate(prob1)

    # --- 6) 時系列整合：予測は data の [seq_len-1:] に対応 ---
    effective = data.iloc[seq_len-1:].copy()
    effective = effective.reset_index(drop=True)
    assert len(effective) == len(preds)

    # --- 7) 出力DF構築（OHLCV, Pred, Label） ---
    # df_raw から同じ timestamp に一致する OHLCV を紐づける
    # 前提: df_raw に 'timestamp','O','H','L','C','V' がある
    base = df_raw.copy()
    base['timestamp'] = pd.to_datetime(base['timestamp'])
    out = effective[['timestamp']].merge(
        base[['timestamp', 'O','H','L','C','V']],
        on='timestamp',
        how='left'
    )

    out['Pred']  = preds.astype(int)
    out['Label'] = effective[label_col].astype(int).values
    if keep_prob:
        out['ProbLong'] = prob1.astype(float)

    # 列順を明示
    cols = ['timestamp','O','H','L','C','V','Pred','Label']
    if keep_prob:
        cols.append('ProbLong')
    df_predicted = out[cols].copy()

    return df_predicted


# === 実行例 ===
# すでに上のセルで `df_raw`, `model_path`, `scaler_path`, `Features`, `Label`, `LSTM_lookback`, `device` が定義済みの想定
df_predicted = run_inference_build_df(
    df_raw=df_raw,
    model_path=model_path,
    scaler_path=scaler_path,
    features=Features,
    label_col=Label,
    seq_len=LSTM_lookback,
    device=device,
    batch_size=LSTM_batch,
    keep_prob=False,          # 確信度が欲しければ True
)


In [ ]:
#11. VWAP Regime + Position/Trade columns
import numpy as np
import pandas as pd

def build_vwap_regime_and_trades(
    df_pred: pd.DataFrame,
    vwap_short: int = 20,
    vwap_medium: int = 60,
    price_col: str = "C",
    use_typical_price: bool = False,  # Trueなら (H+L+C)/3 を使用
) -> pd.DataFrame:
    """
    入力: df_pred (timestamp, O,H,L,C,V, Pred を含む)
    出力: df_out = df_pred に [VWAP_short, VWAP_medium, Regeme, Position, Trade] を付与
    仕様:
      1) VWAP_short/medium は Rolling(window) の (Price*V).sum / V.sum
      2) Regeme: VWAP_s > VWAP_m → 1 / VWAP_s < VWAP_m → 0 / 同値やNaNは NaN
      3) Position:
           - 先頭は NaN で開始
           - Pred と Regeme が初めて一致したら、その値(0/1)でポジション開始
           - 以降 Pred==Regeme かつ 現在ポジションと異なるときだけ 0/1 を入れ替え
           - それ以外は直前のポジションを維持
      4) Trade:
           - 基本 NaN
           - Position が切り替わったバーだけ 0/1 を記録（初回開始時も記録）
    """
    df = df_pred.copy()

    # --- 価格系列 ---
    if use_typical_price:
        price = (df['O'] + df["H"] + df["L"] + df["C"]) / 4.0
    else:
        price = df[price_col].astype(float)

    vol = df["V"].astype(float)

    # --- Rolling VWAP ---
    pv = price * vol
    vwap_s = pv.rolling(vwap_short, min_periods=1).sum() / vol.rolling(vwap_short, min_periods=1).sum()
    vwap_m = pv.rolling(vwap_medium, min_periods=1).sum() / vol.rolling(vwap_medium, min_periods=1).sum()

    # 先頭の安定化: 窓が埋まってない期間は NaN にしたいなら下記を有効化
    vwap_s = vwap_s.where(vol.rolling(vwap_short).count() >= vwap_short, np.nan)
    vwap_m = vwap_m.where(vol.rolling(vwap_medium).count() >= vwap_medium, np.nan)

    df["VWAP_short"]  = vwap_s
    df["VWAP_medium"] = vwap_m

    # --- Regeme: 1/0/NaN (同値はNaNにしてスイッチ抑制) ---
    reg = np.where(
        (df["VWAP_short"] > df["VWAP_medium"]),
        1,
        np.where(
            (df["VWAP_short"] < df["VWAP_medium"]),
            0,
            np.nan
        )
    )
    df["Regeme"] = reg.astype(float)  # NaNを含むため float 型

    # --- Position の構築（状態機械） ---
    n = len(df)
    pos = np.full(n, np.nan, dtype=float)

    have_started = False
    current = np.nan

    pred = df["Pred"].to_numpy(dtype=float)  # 0/1 想定（NaNも許容）
    rege = df["Regeme"].to_numpy(dtype=float)

    for i in range(n):
        p = pred[i]
        r = rege[i]

        # どちらかが NaN → スイッチしない
        if np.isnan(p) or np.isnan(r):
            if have_started:
                pos[i] = current
            else:
                pos[i] = np.nan
            continue

        # まだ開始していない → 一致したら開始
        if not have_started:
            if p == r:
                current = p
                have_started = True
                pos[i] = current
            else:
                pos[i] = np.nan
            continue

        # 既に開始済み → 一致して、かつ現在と違えば切替
        if p == r and current != p:
            current = p

        pos[i] = current

    df["Position"] = pos  # 0/1/NaN

    # --- Trade: Position が切り替わるポイントだけ 0/1、それ以外 NaN ---
    # 先頭開始も「切替」とみなす
    position_series = pd.Series(df["Position"])
    changed = position_series.ne(position_series.shift(1))  # 変更フラグ（NaN比較もTrueになる）
    # ただし「両方NaN」のときは変更ではないので補正
    both_nan = position_series.isna() & position_series.shift(1).isna()
    changed = changed & ~both_nan

    trade = position_series.where(changed, np.nan)
    df["Trade"] = trade

    # 列順を少し整えて返す（任意）
    desired_order = [
        "timestamp", "O","H","L","C","V",
        "Pred", "Label" if "Label" in df.columns else None,
        "VWAP_short","VWAP_medium","Regeme","Position","Trade"
    ]
    desired_order = [c for c in desired_order if c is not None]
    df = df[desired_order]

    return df

# === 使い方例 ===
# df_rule = build_vwap_regime_and_trades(df_pred, vwap_short=20, vwap_medium=60, price_col="C", use_typical_price=False)
# display(df_rule.head(15))
